In [1]:
from typing import cast
from bitcoinlib.wallets import Wallet, WalletError, wallet_delete_if_exists
from bitcoinlib.keys import HDKey
import random

# Create a Multisig wallet with 2 cosigners which both need to sign a transaction.

NETWORK = 'testnet'
k1 = HDKey('tprv8ZgxMBicQKsPd1Q44tfDiZC98iYouKRC2CzjT3HGt1yYw2zuX2awTotzGAZQEAU9bi2M5MCj8iedP9MREPjUgpDEBwBgGi2C8eK'
           '5zNYeiX8', network=NETWORK)
k2 = HDKey('tprv8ZgxMBicQKsPeUbMS6kswJc11zgVEXUnUZuGo3bF6bBrAg1ieFfUdPc9UHqbD5HcXizThrcKike1c4z6xHrz6MWGwy8L6YKVbgJ'
           'MeQHdWDp', network=NETWORK)

wallet_delete_if_exists('multisig_2of2_cosigner1')
wallet_delete_if_exists('multisig_2of2_cosigner2')

w1 = cast(Wallet, Wallet.create('multisig_2of2_cosigner1', sigs_required=2,
                    keys=[k1, k2.public_master(multisig=True)], network=NETWORK))

w2 = cast(Wallet, Wallet.create('multisig_2of2_cosigner2',  sigs_required=2,
                    keys=[k1.public_master(multisig=True), k2], network=NETWORK))

print(w1.get_key())
print(w2.get_key())

<WalletKey(key_id=69, name=Multisig Key 68/52, wif=multisig-<Address(address=2NBrLTapyFqU4Wo29xG4QeEt8kn38KVWRRr)>, path=m/45'/0/0/0)>
<WalletKey(key_id=73, name=Multisig Key 61/72, wif=multisig-<Address(address=2NBrLTapyFqU4Wo29xG4QeEt8kn38KVWRRr)>, path=m/45'/0/0/0)>


In [2]:
# An address starting with "2": a testnet p2sh address
print("Deposit testnet bitcoin to this address to create transaction: ", w1.get_key().address)

Deposit testnet bitcoin to this address to create transaction:  2NBrLTapyFqU4Wo29xG4QeEt8kn38KVWRRr


In [3]:
# Create a transaction in the first wallet

w1.utxos_update()
# send from UTXO to 'mwCwTceJvYV27KXBc3NJZys6CjsgsoeHmf'
# SHOULD send but there is not enough fee
t = w1.sweep('mwCwTceJvYV27KXBc3NJZys6CjsgsoeHmf', min_confirms=0)
t.info()

WalletError: Amount to send is smaller then dust amount: -125

In [ ]:
# And then import the transaction in the second wallet, sign it and push it to the network

# w2.get_key()
t2 = w2.transaction_import(t)
t2.sign()

t2.send() # SHOULD send, but there is not enough fee
t2.info()